<a href="https://colab.research.google.com/github/ncadsxhvs/869_course/blob/main/jf_randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

polynomial encoder for geo code
auto feat
feature engineer  
multi class neural network 

dmg serverity 1-2 is more than 2-3, how to add weight on that




In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install category_encoders
!pip install ipyparallel


# a = 1
# with open('drive/MyDrive/Colab Notebooks/eggs.csv', 'w', newline='') as csvfile:
#     spamwriter = csv.writer(csvfile, delimiter=' ',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerow([a] * 5 
#                         + ['Baked Beans'])

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 386 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 298 kB 4.1 MB/s 
     |████████████████████████████████| 1.6 MB 46.7 MB/s 


In [158]:
import pandas as pd
import numpy as np
import os
import sys

# To plot pretty figures
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 15


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, f1_score, make_scorer, accuracy_score, mean_squared_error, roc_curve, plot_roc_curve, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PolynomialFeatures, RobustScaler, OrdinalEncoder#, autofeat
from scipy.stats import reciprocal, uniform, randint

from sklearn.decomposition import PCA
import category_encoders as ce


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import xgboost as xgb

#from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import ShuffleSplit
import csv
from sklearn.model_selection import ShuffleSplit

In [159]:
random_state = 42
np.random.seed(random_state)

In [160]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=16)
with cluster as rc:
    ar = rc[:].apply_async(os.getpid)
    pid_map = ar.get_dict()


Using existing profile dir: '/root/.ipython/profile_default'


INFO:ipyparallel.cluster.cluster.1669334984-0gml:Using existing profile dir: '/root/.ipython/profile_default'


Starting 16 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


INFO:ipyparallel.cluster.cluster.1669334984-0gml:Starting 16 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/16 [00:00<?, ?engine/s]

...

INFO:ipyparallel.cluster.cluster.1669334984-0gml:engine set stopped 1669334986: {'engines': {'0': {'exit_code': 0, 'pid': 1656, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 1658, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 1660, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 1662, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 1666, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 1668, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 1670, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 1672, 'identifier': '7'}, '8': {'exit_code': 0, 'pid': 1676, 'identifier': '8'}, '9': {'exit_code': 0, 'pid': 1680, 'identifier': '9'}, '10': {'exit_code': 0, 'pid': 1685, 'identifier': '10'}, '11': {'exit_code': 0, 'pid': 1689, 'identifier': '11'}, '12': {'exit_code': 0, 'pid': 1691, 'identifier': '12'}, '13': {'exit_code': 0, 'pid': 1693, 'identifier': '13'}, '14': {'exit_code': 0, 'pid': 1699, 'identifier': '14'}, '15': {'exit_code': 0, 'pid': 1706, 'identifier': '15'}}, 'exit_code': 0

Stopping controller


INFO:ipyparallel.cluster.cluster.1669334984-0gml:Stopping controller


Controller stopped: {'exit_code': 0, 'pid': 1621, 'identifier': 'ipcontroller-1669334984-0gml-75'}


INFO:ipyparallel.cluster.cluster.1669334984-0gml:Controller stopped: {'exit_code': 0, 'pid': 1621, 'identifier': 'ipcontroller-1669334984-0gml-75'}


In [161]:
def get_roc_scores(model, X, y):
    y_scores = model.predict_proba(X)[:, 1]
    fpr, tpr, thresholds = roc_curve(y, y_scores)
    i = np.argmax(tpr - fpr)
    return fpr, tpr, thresholds, i, y_scores

def plot_roc_threshold_curves(model, X, y, modelName = "Default"):
    fpr, tpr, thresholds, i, y_scores = get_roc_scores(model, X, y)
    print(modelName,"ROC AUC Score -",round(roc_auc_score(y,y_scores),3))
    plt.plot(fpr, tpr, linewidth=4, label=modelName)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.legend(loc="lower right", fontsize=16)
    plt.scatter(fpr[i], tpr[i], c='r', s=100)
    plt.title('ROC Curve');

In [449]:
root_path = "drive/MyDrive/Colab Notebooks/"
train = root_path + "Train_Values.csv"
target = root_path + 'Train_Labels.csv'
test = root_path + 'Test_Values.csv'

In [450]:
data_orig = pd.read_csv(train)[:10000]
data_target = pd.read_csv(target)[:10000]
data_target['damage_grade'] = data_target['damage_grade'].apply(lambda x: x-1)
data_target.damage_grade.unique()
data_orig_target = pd.merge(data_orig,data_target, on = 'building_id')
data_test = pd.read_csv(test)
X_test = data_test
X_test.drop(axis = 1, columns = 'building_id', inplace = True)

Geo location clustering

In [451]:
# data_orig.columns
data_orig_target['geo_level_1_id'] = data_orig_target['geo_level_1_id'].astype('object')
data_orig_target['geo_level_2_id'] = data_orig_target['geo_level_2_id'].astype('object')
data_orig_target['geo_level_3_id'] = data_orig_target['geo_level_3_id'].astype('object')

geo_set = data_orig_target
# .sample(frac=0.05, replace=True, random_state=random_state)
# geo_features = geo_set[[ 'geo_level_1_id', 'geo_level_2_id','geo_level_3_id', 'damage_grade']]
geo_features = geo_set[[ 'geo_level_1_id', 'geo_level_2_id','geo_level_3_id', 'damage_grade']]
# geo_features.count

Cleaning and Preprocessing (10%):
What cleaning and preprocessing steps did you try? Which worked, which didn’t?

Drop Building ID

In [452]:
from sklearn.cluster import KMeans
# create kmeans model/object
kmeans = KMeans(
    init="random",
    n_clusters=16,
    n_init=10,
    max_iter=300,
    random_state=random_state
)

In [453]:
kmeans.fit(geo_features)


KMeans(init='random', n_clusters=16, random_state=42)

In [454]:
data_orig_target.drop(axis = 1, columns = ['building_id'], inplace = True)

data_clean = data_orig_target



In [455]:
labels = kmeans.labels_
data_clean['cluster'] = labels
_clusters = data_clean.groupby('cluster')['cluster'].count()
# print(_clusters)
data_clean.head()


,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,cluster
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,2,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,1,14
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,2,15
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,1,1
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,2,7


In [456]:
# data_clean['geo_level_1_id'] = data_clean['geo_level_1_id'].astype('object')
# data_clean['geo_level_2_id'] = data_clean['geo_level_2_id'].astype('object')
# data_clean['geo_level_3_id'] = data_clean['geo_level_3_id'].astype('object')

In [457]:
# for (columnName, columnData) in data_clean.iteritems():
#     print(columnName+' (',columnData.dtype,"): ", set(columnData.values[:2]))

Data shuffling


## Feature engineering (10%)
•	What feature engineering and selection steps did you try?\
•	Which worked, which didn’t?


In [458]:
X = data_clean.drop("damage_grade", axis = 1)
y = data_clean["damage_grade"].copy()
X_cp = X
X['lowHeight'] = X['height_percentage'].map(lambda s: 1 if s <= 5 else 0)
X['medHeight'] = X['height_percentage'].map(lambda s: 1 if 7 <= s <= 8 else 0)
X_test['lowHeight'] = X_test['height_percentage'].map(lambda s: 1 if s <= 5 else 0)
X_test['medHeight'] = X_test['height_percentage'].map(lambda s: 1 if 7 <= s <= 8 else 0)

In [459]:
# X['height_percentage']
print(X_cp["height_percentage"])

0       5
1       7
2       5
3       5
4       9
       ..
9995    5
9996    5
9997    4
9998    5
9999    5
Name: height_percentage, Length: 10000, dtype: int64


In [460]:
encoder = ce.PolynomialEncoder(cols=["height_percentage"])
encoder.fit_transform(X_cp, verbose=1).iloc[:,8:14].head()
X = X_cp

/usr/local/lib/python3.7/dist-packages/category_encoders/base_contrast_encoder.py:127: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  category=FutureWarning)


radom split

In [461]:
# rs = ShuffleSplit(n_splits=5, test_size=.25, random_state=random_state)
# rs.get_n_splits(X)
# print(rs)
# for train_index, test_index in rs.split(X):
#   print("TRAIN:", train_index, "TEST:", test_index, len(train_index))

Feature engineering (10%)
• What feature engineering and selection steps did you try?
• Which worked, which didn’t?

In [462]:
numeric_cols = list(X.select_dtypes("int").columns)


In [463]:
cat_cols = list(X.select_dtypes("object").columns)


In [464]:
rf_clf = RandomForestClassifier(random_state =random_state )

In [465]:
num_pipeline = Pipeline([
            ('std_scaler', StandardScaler()),
    ])

In [466]:
geo_pipeline = Pipeline([
    ("target_encoder", ce.TargetEncoder()),
    ])

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [467]:
cat_pipeline = Pipeline([
    ("target_encoder", ce.TargetEncoder()),
    ])

In [468]:
preprocessor = Pipeline([
    ('ct',ColumnTransformer(
        transformers=[
            ('num', num_pipeline,numeric_cols),
            #('geo',geo_pipeline,geo_cols),
            ('cat',cat_pipeline,cat_cols)],
        remainder = 'passthrough',
        sparse_threshold = 0)),#Look into this
    #('pca', PCA(n_components=10))
])

In [469]:
rf_clf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf_clf',rf_clf)
])

In [470]:
#Have to name the model before indicating hyperparameter (see example below)
param_grid = {'rf_clf__n_estimators': [500],
              'rf_clf__criterion': ["gini", "entropy"],
              'rf_clf__min_samples_split': [6],
              'rf_clf__max_features' : [6]
             }

rand_search_rf_clf = RandomizedSearchCV(rf_clf_pipe, 
                                   param_grid, 
                                   cv=3 ,
                                   scoring= 'f1_micro', 
                                   n_jobs = -1, 
                                   verbose = True,
                                 return_train_score=True)

In [471]:
%time rand_search_rf_clf = rand_search_rf_clf.fit(X, y)
# full/500


/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/loc

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


CPU times: user 7.06 s, sys: 53.2 ms, total: 7.12 s
Wall time: 38.7 s


In [472]:
print(rand_search_rf_clf.best_score_)

0.68409927375461


In [365]:
print(rand_search_rf_clf.best_params_)

{'rf_clf__n_estimators': 500, 'rf_clf__min_samples_split': 6, 'rf_clf__max_features': 6, 'rf_clf__criterion': 'gini'}


In [366]:
rand_search_rf_clf

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              Pipeline(steps=[('ct',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 sparse_threshold=0,
                                                                                 transformers=[('num',
                                                                                                Pipeline(steps=[('std_scaler',
                                                                                                                 StandardScaler())]),
                                                                                                ['geo_level_1_id',
                                                                                                 'geo_level_2_id',
                                           

In [367]:
rf_clf_pred = rand_search_rf_clf.predict(X_test)

KeyError: ignored

In [ ]:
rf_clf_pred_sub = pd.DataFrame()
data_test = pd.read_csv(test)
rf_clf_pred_sub['building_id']= data_test['building_id']
rf_clf_pred_sub['damage_grade'] = rf_clf_pred+1
print(rf_clf_pred_sub)
rf_clf_pred_sub.to_csv(root_path + 'rf_full_500_20.csv',index=False)